In [3]:
from imutils import paths
import face_recognition
import pickle
import cv2
import os
import time
import numpy as np

In [4]:
show_detected_faces = False
imagePaths = list(paths.list_images("..\\data\\training"))

print("Images# : %s" % (len(imagePaths),))
face_encodings = open("..\\data\\face_enc.tsv", "w")

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    name = imagePath.split(os.path.sep)[-2]
    print(imagePath)
       
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    boxes = face_recognition.api.face_locations(rgb, model='hog')
    
    # show face boxes found
    if show_detected_faces or len(boxes) != 1:
        for face_box in boxes:
            print(face_box)
            y1, x2, y2, x1 = face_box
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.imshow("Frame", image)
        cv2.waitKey(0)
    
    if len(boxes) != 1:
        print ("One face per image is expected, found : " + str(len(boxes)))
        break

    encodings = face_recognition.api.face_encodings(rgb, boxes)

    for i in range(0, len(encodings)):       
        encoding_data = map(str, encodings[i])
        face_encodings.write(name + "\t" + "|".join(encoding_data) + "\t" + str(boxes[i]) + "\t" + imagePath + "\n")
        
face_encodings.close()


Images# : 13
..\data\training\Gal Gadot\01.jpg
..\data\training\Gal Gadot\02.jpg
..\data\training\Gal Gadot\03.jpg
..\data\training\Gal Gadot\04.jpg
..\data\training\Gal Gadot\05.jpg
..\data\training\Ryan Reynolds\reynolds.jpg
..\data\training\Ryan Reynolds\Ryan-Reynolds-2011.jpg
..\data\training\Ryan Reynolds\Ryan-Reynolds.png
..\data\training\Ryan Reynolds\Ryan_Reynolds 02.jpg
..\data\training\Ryan Reynolds\Ryan_Reynolds.jpg
..\data\training\The Rock\02.jpg
..\data\training\The Rock\03.jpg
..\data\training\The Rock\4608.jpg


In [5]:
class face_match:
    name = ""
    distance = 0
    def __init__ (self, n, d):
        self.name = n
        self.distance = d

cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPathface)

# load the face lookup data
names_lookup = []
face_lookup = []
with open("..\\data\\face_enc.tsv", "r") as face_data:
    for line in face_data:
        tokens = line.split("\t")
        names_lookup.append(tokens[0])
        face_data = tokens[1].split("|")
        face_floats = np.array(face_data).astype(float)
        face_lookup.append(face_floats)


#img_path = "..\\data\\maxresdefault.jpg"
#img_path = "..\\data\\training\\Ryan Reynolds\\Ryan_Reynolds.jpg"
img_path = "..\\data\\group02.jpg"
image = cv2.imread(img_path)
print(image.shape)

rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
faces = face_recognition.api.face_locations(rgb, number_of_times_to_upsample=1)

encodings = face_recognition.api.face_encodings(rgb, known_face_locations=faces)
face_results = []
for enc_index in range(0, len(encodings)):
    encoding = encodings[enc_index]
    face_distances = face_recognition.api.face_distance(face_lookup, encoding)

    closest_face = None
    for i in range(0, len(face_distances)):
        if face_distances[i] < 0.55 and (closest_face is None or closest_face.distance > face_distances[i]):
            closest_face = face_match(names_lookup[i], face_distances[i])
            
    face_results.append(closest_face)

# process the result
# draw some boxes on the faces with our matched face name
for i in range(0, len(face_results)):
    face_box = faces[i]
    face_guess = face_results[i]
    
    y1, x2, y2, x1 = face_box
    
    colour = (0, 0, 255)
    if face_guess is not None:
        colour = colour = (0, 255, 0)
    
    cv2.rectangle(image, (x1, y1), (x2, y2), colour, 2)
    if face_guess is not None:
        cv2.putText(image, face_guess.name, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.50, colour, 1)
        cv2.putText(image, str(round(face_guess.distance, 5)), (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.50, colour, 1)

cv2.imshow("Frame", image)
cv2.waitKey(0)

(507, 762, 3)


-1